In [66]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
np.random.seed(1337)
random.seed(1337)

In [68]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

In [69]:
n = MLP(3, [1, 1])
m = MLP(3, [3, 1])
n, m

(MLP of [Layer of [ReLUNeuron(3)], Layer of [LinearNeuron(1)]],
 MLP of [Layer of [ReLUNeuron(3), ReLUNeuron(3), ReLUNeuron(3)], Layer of [LinearNeuron(3)]])

In [74]:
class _: # MLP
    from typing import List
    def __init__(self, nin: int, nouts: List[int]):
        sz = [nin] + nouts
        self.layers = [
            Layer(sz[i], sz[i + 1], nonlin=i != len(nouts) - 1)
            for i in range(len(nouts))
        ]
# MLP CONSTRUCTOR
# this is straightforward right, we make layers of
# cin, cout combos but we only make #cout of them
# each layers takes nin starting with nin, but then
# all other layers take the nouts[i] of the previous

# LINEAR OUTPUT
# the last layers is linear so we can predict any number not just [0,int)

# RELU LAYERS
# ReLU (Rectified Linear Unit) introduces non-linearity into neural networks
# due to its non-linear nature. The ReLU function is defined as:
#
# f(x) = max(0, x)
#
# without relus (or any non linear activation) nn's would be linear funtions

#  By definition, the ReLU is 𝑚𝑎𝑥(0,𝑥). Therefore, if we split the domain from
# (−∞,0] or [0,∞), then the function is linear. However, it's easy to see
# that 𝑓(−1)+𝑓(1)≠𝑓(0). Hence, by definition, ReLU is not linear.


In [70]:
def internals(n: MLP) -> None:
  for layer in n.layers:
    print(layer, '---')
    for neuron in layer.neurons:
      print(neuron, '*')
      for value in neuron.parameters(): # .w and [.b]
        print(value, '.')

In [71]:
internals(n)
# MLP Diagram:
#
#   Input (3 features)
#       ↓
# Layer 1 (ReLUNeuron) # w: [0.2, 0.1, 0.3], b: 0.0
#       ↓
# Layer 2 (LinearNeuron) # w: [0.2], b: 0.0
#       ↓
#  Output (1 output)

Layer of [ReLUNeuron(3)] ---
ReLUNeuron(3) *
Value(data=0.23550571390294128, grad=0) .
Value(data=0.06653114721000164, grad=0) .
Value(data=-0.26830328150124894, grad=0) .
Value(data=0, grad=0) .
Layer of [LinearNeuron(1)] ---
LinearNeuron(1) *
Value(data=0.1715747078045431, grad=0) .
Value(data=0, grad=0) .


In [72]:
internals(m)

Layer of [ReLUNeuron(3), ReLUNeuron(3), ReLUNeuron(3)] ---
ReLUNeuron(3) *
Value(data=-0.6686254326224383, grad=0) .
Value(data=0.6487474938152629, grad=0) .
Value(data=-0.23259038277158273, grad=0) .
Value(data=0, grad=0) .
ReLUNeuron(3) *
Value(data=0.5792256498313748, grad=0) .
Value(data=0.8434530197925192, grad=0) .
Value(data=-0.3847332240409951, grad=0) .
Value(data=0, grad=0) .
ReLUNeuron(3) *
Value(data=0.9844941451716409, grad=0) .
Value(data=-0.5901079958448365, grad=0) .
Value(data=0.31255526637777775, grad=0) .
Value(data=0, grad=0) .
Layer of [LinearNeuron(3)] ---
LinearNeuron(3) *
Value(data=0.8246106857787521, grad=0) .
Value(data=-0.7814232047574572, grad=0) .
Value(data=0.6408752595662697, grad=0) .
Value(data=0, grad=0) .


In [73]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

